In [1]:
# ================= CELL TRAINING: FINE-TUNING SBERT =================
from sentence_transformers import SentenceTransformer, InputExample, losses, models
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from torch.utils.data import DataLoader
import pandas as pd
import os
import math

# --- 1. KONFIGURASI ---
# Pilih salah satu model dari folder lokalmu
# Ganti path ini setiap kali mau ganti model yang dilatih
BASE_MODEL_PATH = r"D:\71210730\Versi 4\Model\hf_models\LazarusNLP__all-nusabert-base-v4"

# Output akan disimpan di folder baru
OUTPUT_PATH = r"D:\71210730\Versi 4\Model\hf_models\Trained_SBERT\finetuned_all-nusabert-base_v1"

# Hyperparameters (Standar SBERT)
BATCH_SIZE = 16 # Kalau GPU Memory Error, turunkan jadi 16
EPOCHS = 4
LEARNING_RATE = 2e-5

# --- 2. FUNGSI DETEKSI PREFIX (PENTING BUAT nusabert-base) ---
def get_prefix(model_name_or_path):
    # Cek kalau nama model mengandung 'nusabert-base' atau 'nusabert-base'
    if "nusabert-base" in model_name_or_path.lower():
        print("⚠️ Model nusabert-base Terdeteksi! Menambahkan prefix 'query: ' ke data training.")
        return "query: "
    return ""

prefix = get_prefix(BASE_MODEL_PATH)

# --- 3. PREPARE DATALOADER ---
print("Membaca data CSV...")
train_df = pd.read_csv("train_sbert.csv", encoding='utf-8-sig')
val_df = pd.read_csv("val_sbert.csv", encoding='utf-8-sig')

# Fungsi konversi DataFrame ke InputExample SBERT
def convert_to_examples(df, prefix_str=""):
    examples = []
    for _, row in df.iterrows():
        # Konversi text ke string jaga-jaga
        t1 = prefix_str + str(row['text_1'])
        t2 = prefix_str + str(row['text_2'])
        score = float(row['score_norm']) # Wajib float 0.0 - 1.0
        examples.append(InputExample(texts=[t1, t2], label=score))
    return examples

print("Mengkonversi data ke format InputExample...")
train_examples = convert_to_examples(train_df, prefix)
val_examples = convert_to_examples(val_df, prefix)

# DataLoader
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=BATCH_SIZE)

# --- 4. SETUP MODEL & LOSS ---
print(f"Loading Base Model: {BASE_MODEL_PATH}")
model = SentenceTransformer(BASE_MODEL_PATH)

# Loss Function untuk STS (Cosine Similarity Loss)
# Ini cocok dengan data skor float 0.0 - 1.0
train_loss = losses.CosineSimilarityLoss(model)

# --- 5. SETUP EVALUATOR ---
# Evaluator akan mengecek performa model setiap habis 1 epoch
# Menggunakan Spearman Correlation (Sesuai Bab 2 kamu)
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    val_examples, 
    name='sts-val',
    write_csv=True
)

# --- 6. MULAI TRAINING ---
print(f"Mulai Training selama {EPOCHS} epoch...")
print(f"Model akan disimpan di: {OUTPUT_PATH}")

# Kalkulasi langkah pemanasan (warmup steps) - biasanya 10% dari total steps
warmup_steps = math.ceil(len(train_dataloader) * EPOCHS * 0.1)

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=EPOCHS,
    evaluation_steps=1000,     # Evaluasi setiap 1000 batch (opsional)
    warmup_steps=warmup_steps,
    output_path=OUTPUT_PATH,
    save_best_model=True,      # Simpan model terbaik berdasarkan Val Score
    optimizer_params={'lr': LEARNING_RATE} 
)

print("\n✅ TRAINING SELESAI!")
print(f"Model terbaik tersimpan di: {OUTPUT_PATH}")

c:\Python\Python3.11.9\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


⚠️ Model nusabert-base Terdeteksi! Menambahkan prefix 'query: ' ke data training.
Membaca data CSV...
Mengkonversi data ke format InputExample...


No sentence-transformers model found with name D:\71210730\Versi 4\Model\hf_models\LazarusNLP__all-nusabert-base-v4. Creating a new one with mean pooling.


Loading Base Model: D:\71210730\Versi 4\Model\hf_models\LazarusNLP__all-nusabert-base-v4
Mulai Training selama 4 epoch...
Model akan disimpan di: D:\71210730\Versi 4\Model\hf_models\Trained_SBERT\finetuned_all-nusabert-base_v1


Step,Training Loss,Validation Loss,Sts-val Pearson Cosine,Sts-val Spearman Cosine
1000,0.028600,No log,0.899453,0.877705
2000,0.025600,No log,0.909159,0.889398
3000,0.025300,No log,0.906393,0.886598
4000,0.017500,No log,0.914545,0.895767
5000,0.019000,No log,0.915133,0.896964
6000,0.018700,No log,0.917768,0.899447
7000,0.013300,No log,0.917036,0.900540
8000,0.013100,No log,0.916913,0.899704
9000,0.013600,No log,0.918579,0.901273
10000,0.010400,No log,0.917997,0.902085



✅ TRAINING SELESAI!
Model terbaik tersimpan di: D:\71210730\Versi 4\Model\hf_models\Trained_SBERT\finetuned_all-nusabert-base_v1
